<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_SingleChart_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

For those who may not have access to a live MongoDB instance, we have set up a mechanism where the JSON file that would have been loaded into a hosted MongoDB instance, will be loaded into a locally (that is in the VM) installed MongoDB instance and the queries can be run in a manner that is identical to a live, hosted MongoDB instance

#SetUp Environment

##Install Software

In [1]:
!pip -qq install python-docx                #https://python-docx.readthedocs.io/en/latest/
!/usr/bin/python3 -m pip install "pymongo[srv]"
!apt install mongodb > /dev/null            # install local MongoDB server           
#!service mongodb start                      # start Local MongoDB server instance


     |████████████████████████████████| 5.6 MB 719 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 5.3 MB/s 




##Install, import prerequisites

In [2]:
#Utility functions 
from pymongo import MongoClient
import pandas as pd
import dateutil
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from datetime import timedelta
import pytz

#import matplotlib.pyplot as plt
#import math


In [3]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py 
# ---------------------------

#!wget -O Saraswati.png -q https://github.com/prithwis/parashar21/raw/main/images/Saraswati02.png                 
#!wget -O p21logo.png -q https://github.com/prithwis/parashar21/raw/main/images/p21logo-002-1.png                  

In [4]:
#!cat p21utilsR.py

In [5]:
#!cat p21utils.py
import p21
import p21utils
import p21utilsR
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21utils)
importlib.reload(p21utilsR)

imported p21
p21utils imported
p21utilsR imported
imported p21
p21utils imported
p21utilsR imported


<module 'p21utilsR' from '/content/p21utilsR.py'>

##Get Remote MongoDB credentials

In [6]:
from google.colab import drive
drive.mount('/content/drive')
# -------------------------------------------
# copy credentials file
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/clevercloudMongoDB.py credentials.py
!cp /content/drive/'My Drive'/Praxis/WebCredentials/parashar21URI.py credentials.py
# -------------------------------------------
# MongoDB credentials are stored here
#
#from credentials import rhostName,ruserName,rpassWord,rdbName,rMongoURI
#print(rdbName)
from credentials import parashar21URI

# -------------------------------------------
#mongoURI = rMongoURI                                   # for remote Clever Cloud server
mongoURI = parashar21URI                                   # for remote ATlas server

client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']
# ----------------------------------------------
# Show existing Collections

Mounted at /content/drive


['Parashar21', 'admin', 'local']

In [7]:
#mongoURI = "mongodb://localhost:27017/"                               # for remote server
#client = MongoClient(mongoURI)

db = client.Parashar21
pd.DataFrame(list(db.list_collections()))


,name,type,options,info,idIndex
0,khona21Test,collection,{},"{'readOnly': False, 'uuid': b'&t\xaa\x86\x17=B...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_'}"
1,khona21,collection,{},"{'readOnly': False, 'uuid': b""'\x9e\x81\x8b\xf...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_'}"


In [8]:
kollection = db.khona21Test                   # Set the collection to work with
#kollection = db.khona21                   # Set the collection to work with
totalCharts = kollection.count_documents({})
print(totalCharts)

5


## Required ONLY for standalone database

###Download Horoscope data in JSON and load into MongoDB

In [ ]:
# Retrieve JSON file from Github and load in local Mongo
!wget -O peopleData.json -q https://github.com/prithwis/parashar21/raw/main/data/peopleDataTest5.json

#!gdown -O peopleData.json https://drive.google.com/uc?id=1_VBHE7Hyue8t3hdBdLU6lgpnaWhhRVd4  # 39662 documents OLD file, NOT TO BE USED
#!gdown  -O peopleData.json https://drive.google.com/uc?id=1z4JFd-zCaS71dICKIYx46kQGbXPAHq5C  # 39662 documents
!mongoimport --db parashar21Local --collection khona21Test --drop --file peopleData.json

2022-06-19T00:43:05.185+0000	connected to: localhost
2022-06-19T00:43:05.185+0000	dropping: parashar21Local.khona21Test
2022-06-19T00:43:05.222+0000	imported 5 documents


In [ ]:
kollection = db.khona21Test                   # Set the collection to work with
#kollection = db.khona21                   # Set the collection to work with
totalCharts = kollection.count_documents({})
dbSize = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
print(totalCharts, ' documents,',round(dbSize,2), " KB data")

5  documents, 17.93  KB data


#Report Generation Function
Please note that the generated report MS Office doc format **will be stored in the Colab VM drive** and needs to be downloaded to local machine for viewing

In [9]:
def genSingleChartReport(ChartStyle):
      
    chartData = chart.iloc[0]
    RepID = chart.iloc[0]['pid']['name']
    now = datetime.now(pytz.timezone('Asia/Kolkata'))
    fileName = 'p21_'+RepID+'_'+ChartStyle+'_'+now.strftime("%H%M%S")+'.doc'

    #print(chartData)
    # ---------------------------------------------------------
    #p21utilsR.R01_CreateReportDoc(curQueryStr, printStatus)   # Creates MS Word Doc called p21.document
    p21utilsR.R01A_CreateReportDoc(RepID)                           # Creates MS Word Doc called p21.document
    # ----------------------------------------------------------
    
    p21utilsR.R511_parseChartData(chartData)              # Break up database data into useful pieces
                                            # Creates two primary dicts, GLon, GRet
    for ctype in ['Rashi','Navamsa']:
        p21.ChartType = ctype
        p21utils.R11_LocateGrahaInRashi()   # Converts Graha Long to Rashi positions
                                            # Rashi Num as well as Rashi name
        if ChartStyle == 'Bengal':
            p21utilsR.R12B_drawChart_Bengal()    # Draw Chart in Bengal style
        if ChartStyle == 'South':
            p21utilsR.R12B_drawChart_South()    # Draw Chart in South style
        if ChartStyle == 'North':
            p21utilsR.R12B_drawChart_North()    # Draw Chart in North style
        
    p21utilsR.R512_FormatPage()
    
    # ------------------------------------
    p21.document.save(fileName)
    


#Retrieve Charts 
Based on selective queries <br>
All output messages suppressed <br>
Reports will be available as DOC files in the VM drive <br>
Please specify style of report, 'Bengal', 'North' or 'South' while calling the report generation routine <br>
genSampleReport('Bengal',Sample = False), specifying Sample='False' will stop sampling and print reports for all charts retrieved. Otherwise, by default, only 5 charts, chosen at random will be printed in the DOC file


In [10]:
c = kollection.find().limit(5)
for crnt in c:
    print(crnt)

{'_id': ObjectId('62ae6e9eec17f90290ebe31a'), 'pid': {'tags': ['Author', 'Homemaker', 'PostGraduate'], 'ck': 'F1965121910:265.522.5788.37', 'chartType': 'Rashi', 'name': 'person2', 'cID': '6VIXS23XHOOM'}, 'GLon': {'La': 310.415, 'Su': 243.701, 'Mo': 201.001, 'Ma': 273.467, 'Me': 222.269, 'Ju': 62.748, 'Ve': 284.966, 'Sa': 318.17, 'Ra': 41.248, 'Ke': 221.248}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': False, 'Ju': True, 'Ve': False, 'Sa': False, 'Ra': False, 'Ke': False}, 'BhavN': [' ', 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'BhavA': [' ', 'Kumbh', 'Meen', 'Mesh', 'Vrish', 'Mithun', 'Karkat', 'Simha', 'Kanya', 'Tula', 'Vrishchik', 'Dhanu', 'Makar'], 'Lord': [' ', 'Sa', 'Ju', 'Ma', 'Ve', 'Me', 'Mo', 'Su', 'Me', 'Ve', 'Ma', 'Ju', 'Sa'], 'LordRashiN': [' ', 11, 3, 10, 10, 8, 7, 9, 8, 10, 10, 3, 11], 'LordRashiA': [' ', 'Kumbh', 'Mithun', 'Makar', 'Makar', 'Vrishchik', 'Tula', 'Dhanu', 'Vrishchik', 'Makar', 'Makar', 'Mithun', 'Kumbh'], 'GrahaLordBhav': {'Su': [7], 

In [11]:
#selCond = {'pid.name': {'$eq': 'person1'}}               # Single Person
selCond = {}                                              # All Charts
selCols = {'_id':0,'pid.name':1, 'pid.cID':1}
#c = kollection.find(selCond,p21.selCols).limit(5)
c = kollection.find(selCond,selCols).limit(5)
for crnt in c:
    print(crnt)

{'pid': {'name': 'person2', 'cID': '6VIXS23XHOOM'}}
{'pid': {'name': 'person1', 'cID': 'T4C6AUTTG58O'}}
{'pid': {'name': 'person3', 'cID': '21RXO5VDC7VQ'}}
{'pid': {'name': 'person4', 'cID': 'WMRUM7H923IZ'}}
{'pid': {'name': 'NDModi', 'cID': 'MM3NRVKU17P3'}}


In [13]:
#selCond = {'pid.name': {'$eq': 'person1'}}               # Single Person
selCond = {"pid.name": {"$regex":"^per"}}                 # Name starting with per
selCols = {'_id':0,'pid.name':1, 'pid.cID':1}
#c = kollection.find(selCond,p21.selCols).limit(5)
chartSet = pd.DataFrame(list(kollection.find(selCond,selCols)))
chartSet

,pid
0,"{'name': 'person2', 'cID': '6VIXS23XHOOM'}"
1,"{'name': 'person1', 'cID': 'T4C6AUTTG58O'}"
2,"{'name': 'person3', 'cID': '21RXO5VDC7VQ'}"
3,"{'name': 'person4', 'cID': 'WMRUM7H923IZ'}"


In [14]:
selCond = {'pid.name': {'$eq': 'person3'}}               # Single Person
chart = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
chart.iloc[0]['pid']['name']
#for key,value in chart.iloc[0]['pid'].items():
#	print(key, ':', value)
#genSampleReport('Bengal',Sample = False)

'person3'

In [15]:
%%capture
genSingleChartReport('Bengal')

#Old Stuff Ignore

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}}       # La aspected by Sa          

                    ]
}

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genSampleReport('North')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}}       # Su conjunct Me 

                    ]
}

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me 
                        {'GrahaBhava.Mo' : {'$eq': 1}}              # Mo in First House

                    ]
}

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me 
                        {'GrahaBhava.Mo' : {'$eq': 1}} ,            # Mo in First House
                        {'LordBhav.4' : {'$eq': 5}}                 # 4th Lord in 5th House

                    ]
}

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal',Sample = False)

In [ ]:
!rm *.doc

In [ ]:
#df = pd.DataFrame(list(kollection.find({'pid.tags': {"$in": ['PostGraduate']}},{'pid.name':1,"_id":0,'GLon':1,'GRet':1})))
#df